In [1]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain-groq \
  langchain-community==0.3.16 \
  langsmith==0.3.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.3/333.3 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.7 requires langchain-core>=1.0.0, but you have langchain-core 0.3.33 which is incompatible.


In [2]:
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "langchain-agents"

Enter LangSmith API Key: ··········


In [3]:
import os
from getpass import getpass
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY") or \
    getpass("Enter your GROQ API key: ")

Enter your GROQ API key: ··········


In [4]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

# Define the multiply tool
@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

# Define the exponentiate tool
@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x


# Creating an Agent

```
 agent = (
      <input parameters, including chat history and user query>
      | <prompt>
      | <LLM with tools>
  )
```

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),    # observation
])

In [6]:
llm = ChatGroq(temperature=0.0, model="llama-3.3-70b-versatile")

In [7]:
from langchain_core.runnables.base import RunnableSerializable

tools = [add, multiply, exponentiate, subtract]

agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")
)

In [8]:
tool_call = agent.invoke({
    "input": "what is 10 + 10", "chat_history": []
})

tool_call

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'wj4hmh5yv', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 504, 'total_tokens': 520, 'completion_time': 0.04729623, 'completion_tokens_details': None, 'prompt_time': 0.049024796, 'prompt_tokens_details': None, 'queue_time': 0.189917041, 'total_time': 0.096321026}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3272ea2d91', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5ce8a5e9-b536-4ac8-a068-96946beb913d-0', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': 'wj4hmh5yv', 'type': 'tool_call'}], usage_metadata={'input_tokens': 504, 'output_tokens': 16, 'total_tokens': 520})

In [9]:
tool_call.tool_calls

[{'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': 'wj4hmh5yv',
  'type': 'tool_call'}]

From here, we have the tool name that our LLM wants to use and the args that it wants to pass to that tool. We can see that the tool add is being used with the arguments x=10 and y=10. The agent.invoke method has not executed the tool function; we need to write that part of the agent code ourselves.

Executing the tool code requires two steps:

1. Map the tool name to the tool function.

2. Execute the tool function with the generated args.

In [10]:
# create tool name to function mapping
name2tool = {tool.name: tool.func for tool in tools}

In [11]:
name2tool

{'add': <function __main__.add(x: float, y: float) -> float>,
 'multiply': <function __main__.multiply(x: float, y: float) -> float>,
 'exponentiate': <function __main__.exponentiate(x: float, y: float) -> float>,
 'subtract': <function __main__.subtract(x: float, y: float) -> float>}

In [12]:
tool_exec_content = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec_content

20

In [14]:
from langchain_core.messages import ToolMessage

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_exec_content}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "what is 3 + 5",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})

out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'sseyjghnf', 'function': {'arguments': '{"x":20,"y":10}', 'name': 'subtract'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 538, 'total_tokens': 554, 'completion_time': 0.031050528, 'completion_tokens_details': None, 'prompt_time': 0.031520308, 'prompt_tokens_details': None, 'queue_time': 0.017037064, 'total_time': 0.062570836}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_bebe2dd4fb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dad1e429-5a09-4d28-b199-c3b09124c38b-0', tool_calls=[{'name': 'subtract', 'args': {'x': 20, 'y': 10}, 'id': 'sseyjghnf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 538, 'output_tokens': 16, 'total_tokens': 554})

Despite having the answer in our agent_scratchpad, the LLM still tries to use the tool again. This behaviour happens because we bonded the tools to the LLM with tool_choice="any". When we set tool_choice to "any" or "required", we tell the LLM that it MUST use a tool, i.e., it cannot provide a final answer.

There's two options to fix this:

1. Set tool_choice="auto" to tell the LLM that it can choose to use a tool or provide a final answer.

2. Create a final_answer tool

In [15]:
# option 1
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="auto")
)

In [16]:
tool_call = agent.invoke({
    "input": "what is 10 + 10", "chat_history": []
})

tool_call

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'wy2zv4cv4', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 488, 'total_tokens': 506, 'completion_time': 0.065976897, 'completion_tokens_details': None, 'prompt_time': 0.025590731, 'prompt_tokens_details': None, 'queue_time': 0.039479292, 'total_time': 0.091567628}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_43d97c5965', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9f827f35-3a4b-4b89-8ec0-f12f4f189847-0', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': 'wy2zv4cv4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 488, 'output_tokens': 18, 'total_tokens': 506})

In [17]:
tool_output = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_output}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out


AIMessage(content='The answer to 10 + 10 is 20.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 522, 'total_tokens': 535, 'completion_time': 0.041161159, 'completion_tokens_details': None, 'prompt_time': 0.027047612, 'prompt_tokens_details': None, 'queue_time': 0.039891745, 'total_time': 0.068208771}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_bebe2dd4fb', 'finish_reason': 'stop', 'logprobs': None}, id='run-18964085-da9c-4936-8453-f2382c0ccb11-0', usage_metadata={'input_tokens': 522, 'output_tokens': 13, 'total_tokens': 535})

In [18]:
# option 2 (Recommended)

@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`.
    """
    return {"answer": answer, "tools_used": tools_used}

In [19]:
tools = [final_answer, add, subtract, multiply, exponentiate]

# we need to update our name2tool mapping too
name2tool = {tool.name: tool.func for tool in tools}

agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
)

In [20]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call.tool_calls

[{'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': '229hrber6',
  'type': 'tool_call'}]

In [21]:
tool_out = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_out}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})

out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'qdd78h9wp', 'function': {'arguments': '{"answer":"The answer to 10 + 10 is 20","tools_used":["add"]}', 'name': 'final_answer'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 700, 'total_tokens': 729, 'completion_time': 0.063691063, 'completion_tokens_details': None, 'prompt_time': 0.075622068, 'prompt_tokens_details': None, 'queue_time': 0.182670171, 'total_time': 0.139313131}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_f8b414701e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7cf9b681-012b-478f-9124-8f9f1834f45f-0', tool_calls=[{'name': 'final_answer', 'args': {'answer': 'The answer to 10 + 10 is 20', 'tools_used': ['add']}, 'id': 'qdd78h9wp', 'type': 'tool_call'}], usage_metadata={'input_tokens': 700, 'output_tokens': 29, 'total_tokens': 729})

In [22]:
out.tool_calls

[{'name': 'final_answer',
  'args': {'answer': 'The answer to 10 + 10 is 20', 'tools_used': ['add']},
  'id': 'qdd78h9wp',
  'type': 'tool_call'}]

In [26]:
out.tool_calls[0]["args"]["answer"]

'The answer to 10 + 10 is 20'

# Building a Custom Agent Execution Loop

In [28]:
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
import json

class CustomAgentExecutor:
    chat_history: list[BaseMessage]

    def __init__(self, max_iterations: int = 3):
        self.chat_history = []
        self.max_iterations = max_iterations
        self.agent: RunnableSerializable = (
            {
                "input": lambda x: x["input"],
                "chat_history": lambda x: x["chat_history"],
                "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
            }
            | prompt
            | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
        )

    def invoke(self, input: str) -> dict:
        # invoke the agent but we do this iteratively in a loop until
        # reaching a final answer
        count = 0
        agent_scratchpad = []
        while count < self.max_iterations:
            # invoke a step for the agent to generate a tool call
            tool_call = self.agent.invoke({
                "input": input,
                "chat_history": self.chat_history,
                "agent_scratchpad": agent_scratchpad
            })
            # add initial tool call to scratchpad
            agent_scratchpad.append(tool_call)
            # otherwise we execute the tool and add it's output to the agent scratchpad
            tool_name = tool_call.tool_calls[0]["name"]
            tool_args = tool_call.tool_calls[0]["args"]
            tool_call_id = tool_call.tool_calls[0]["id"]
            tool_out = name2tool[tool_name](**tool_args)
            # add the tool output to the agent scratchpad
            tool_exec = ToolMessage(
                content=f"{tool_out}",
                tool_call_id=tool_call_id
            )
            agent_scratchpad.append(tool_exec)
            # add a print so we can see intermediate steps
            print(f"{count}: {tool_name}({tool_args})")
            count += 1
            # if the tool call is the final answer tool, we stop
            if tool_name == "final_answer":
                break
        # add the final output to the chat history
        final_answer = tool_out["answer"]
        self.chat_history.extend([
            HumanMessage(content=input),
            AIMessage(content=final_answer)
        ])
        # return the final answer in dict form
        return json.dumps(tool_out)

In [29]:
agent_executor = CustomAgentExecutor()

In [30]:
agent_executor.invoke(input="What is 10 + 10")

0: add({'x': 10, 'y': 10})
1: final_answer({'answer': 'The answer to 10 + 10 is 20.', 'tools_used': ['add']})


'{"answer": "The answer to 10 + 10 is 20.", "tools_used": ["add"]}'

In [31]:
agent_executor.invoke(input="What is 10 * 7.5")

0: exponentiate({'x': 10, 'y': 0})
1: final_answer({'answer': 'The answer to 10 * 7.5 is 75', 'tools_used': ['multiply']})


'{"answer": "The answer to 10 * 7.5 is 75", "tools_used": ["multiply"]}'

In [35]:
agent_executor.invoke(input="What is 10 * 7.5**2")

0: exponentiate({'x': 7.5, 'y': 2})
1: final_answer({'answer': 'The answer to 10 * 7.5^2 is 562.5', 'tools_used': ['exponentiate', 'multiply']})


'{"answer": "The answer to 10 * 7.5^2 is 562.5", "tools_used": ["exponentiate", "multiply"]}'

In [38]:
agent_executor.invoke(input="What is 8 + 5**2")

0: exponentiate({'x': 5, 'y': 2})
1: final_answer({'answer': '33', 'tools_used': ['exponentiate', 'add']})


'{"answer": "33", "tools_used": ["exponentiate", "add"]}'